In [27]:
# TP 01 - MACHINE LEARNING - MAHDI ZEROUAK

In [28]:
# --------------- STEP 1 - LOAD DATA --------------- 

import pandas as pd

# load data into data variable
data = pd.read_csv('Titanic-Dataset.csv')

# print data info
data.info()
# ----- BTW --------- (definition of each variable)
# ** Survived = target variable ** (0 = did not survive) (1 = survived)
# Pclass = ticket class (1st class, 2nd class...etc)
# SibSp = Number of siblings/spouses aboard
# Parch = Number of parents/children aboard
# Fare = ticket price
# Embarked = Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

# PassangerID, Name, Sex, Ticket, Cabin, Age are obvious

# variables that we don't need in our KNN model are [PassengerId, Name, Ticket, Cabin] because they're either just identifiers or text
# --------------------

# print 5 first rows (sample) of our dataset
data.head(5) 

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    str    
 4   Sex          891 non-null    str    
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    str    
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    str    
 11  Embarked     889 non-null    str    
dtypes: float64(2), int64(5), str(5)
memory usage: 83.7 KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [29]:
# ---------------  STEP 2 - PREPROCESS THE DATA --------------- 

# since we have many unnecessary columns we're going to delete them from our data variable that contains the dataset
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)

data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [30]:
# some values are not numbers so we can't compute distance between them (example in sex: male, female)
# solution is to convert these values to numbers

from sklearn.preprocessing import LabelEncoder

# we have two types of enconding

# 1 - label encoding for binary variables like sex: if male then 1 else if female then 0

# 2 - one hot encoding for variables with more than 2 categories, we going to create a new column for each category
# then assign value 1 if the row belongs to that category, else 0

# label encoding for "sex" 
# (1 for male and 0 for female)
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])

# One-hot encoding for "embarked"
# since we have 3 categories, we will create 3 columns
data = pd.get_dummies(data, columns=['Embarked'], dtype=int)

data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


In [31]:
# checking how many null values we have in our dataset
data.isnull().sum()

Survived        0
Pclass          0
Sex             0
Age           177
SibSp           0
Parch           0
Fare            0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [32]:
# we have 177 missed value in age

# we are going to fill them with a median of ages in our dataset
data['Age'] = data['Age'].fillna(data['Age'].median())

data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [33]:
# ---------------  STEP 3 - SPLIT DATA --------------- 

# 70% train, 30% test
from sklearn.model_selection import train_test_split

x = data.drop('Survived', axis=1) 
y = data['Survived']

# split
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.3, # means 30% test and 70% train
    random_state=42, # if we don't specify random_state, each time we run the notebook we will get a different split
                     # which means different test/train data, different accuracy
    stratify=y # since only 38% survived our dataset is imbalanced, 
                # stratify=y will ensure test/train data will be splitted more reliably, not in a random way
)

In [34]:
# ---------------  GOING BACK TO STEP 2 - PREPROCESS THE DATA  --------------- 

# scales numeric features

# since KNN usually use Euclidean distance
# we have to scale because If one feature has a much larger scale (like Fare), it will dominate other variables when calculating
# which means the feature with the largest numeric range controls the model which is not logical 
# scaling ensures that all features contribute equally

# we will not touch binary variables like sex or embarked_c, embarked_q...etc because they can be only 0 or 1

# why we're back to step 2 after finishing with step 3 :
# if we scale on the whole dataset before splitting the data, the scaler can learn the mean of the entire dataset or some other stuff
# that includes our future test data, which is some kind of data leakage
# so CHATGPT said split the data before scaling it, then scale 


# in our case we will scale ('Age', 'Fare', 'SibSp', 'Parch')
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# scaling on train data
# fit_transform(): calculates mean/std AND immediately scales the same data (used in train data)
x_train[['Age', 'Fare', 'SibSp', 'Parch', 'Pclass']] = scaler.fit_transform(x_train[['Age', 'Fare', 'SibSp', 'Parch', 'Pclass']])

# scaling on test data
# transform() does NOT recalculate mean/std — it reuses the ones learned during fit_transform() (used in test data)
x_test[['Age', 'Fare', 'SibSp', 'Parch', 'Pclass']] = scaler.transform(x_test[['Age', 'Fare', 'SibSp', 'Parch', 'Pclass']])

x_train.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
748,-1.589202,1,-0.816093,0.562957,-0.448665,0.465738,0,0,1
45,0.819734,1,-0.126746,-0.474682,-0.448665,-0.478269,0,0,1
28,0.819734,0,-0.126746,-0.474682,-0.448665,-0.481848,0,1,0
633,-1.589202,1,-0.126746,-0.474682,-0.448665,-0.646954,0,0,1
403,0.819734,1,-0.126746,0.562957,-0.448665,-0.314823,0,0,1


In [35]:
# ---------------  STEP 4 - TRAIN THE MODEL --------------- 

from sklearn.neighbors import KNeighborsClassifier

# create KNN model
# K = 3
knn = KNeighborsClassifier(n_neighbors=3)

In [36]:
knn.fit(x_train, y_train)

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",3
,"weights weights: {'uniform', 'distance'}, callable or None, default='uniform'Weight function used in prediction. Possible values:- 'uniform' : uniform weights. All points in each neighborhood are weighted equally.- 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.- [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.Refer to the example entitled:ref:`sphx_glr_auto_examples_neighbors_plot_classification.py`showing the impact of the `weights` parameter on the decisionboundary.",'uniform'
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"p p: float, default=2Power parameter for the Minkowski metric. When p = 1, this is equivalentto using manhattan_distance (l1), and euclidean_distance (l2) for p = 2.For arbitrary p, minkowski_distance (l_p) is used. This parameter is expectedto be positive.",2
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'minkowski'
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.Doesn't affect :meth:`fit` method.",None


In [37]:
# predict labels for test set
y_pred = knn.predict(x_test)

# quick look at first 10 predictions
print(y_pred[:10])

[0 0 0 0 0 0 0 0 0 1]


In [38]:
# ---------------  STEP 5 - EVALUATE THE DATA --------------- 

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# detailed report
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7798507462686567
Confusion Matrix:
 [[139  26]
 [ 33  70]]
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.84      0.82       165
           1       0.73      0.68      0.70       103

    accuracy                           0.78       268
   macro avg       0.77      0.76      0.76       268
weighted avg       0.78      0.78      0.78       268



In [ ]:
# ------------------------------------------------------